In [1]:
from pathlib import Path
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter
import pandas as pd
import os
import math
import scipy.io as sio
from scipy.fftpack import fft,ifft


**First let's load the training data**

Note that the train and test data must be present and the current directory of the notebook. Below is my local address for loading the data.

In [2]:
ROOT_PATH = Path("./train/")
training_data = [(np.load(ROOT_PATH / f"data_{i}.npy"),np.load(ROOT_PATH / f"target_{i}.npy")) for i in range(4)]

**Let's filter the signal to improve the visualisation and get the point that maps to a label**

In [3]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    return butter(order, [lowcut, highcut], fs=fs, btype='band')

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def reshape_array_into_windows(x, sample_rate, window_duration_in_seconds):
    """
    Reshape the data into an array of shape (C, T, window) where 'window' contains
    the points corresponding to 'window_duration' seconds of data.

    Parameters:
    x (numpy array): The input data array.
    sample_rate (int): The number of samples per second.
    window_duration_in_seconds (float): The duration of each window in seconds.

    Returns:
    reshaped_x (numpy array): The reshaped array with shape (C, T, window).
    """
    # Calculate the number of samples in one window
    window_size = int(window_duration_in_seconds * sample_rate)
    
    # Ensure the total length of x is a multiple of window_size
    total_samples = x.shape[-1]
    if total_samples % window_size != 0:
        # Truncate or pad x to make it divisible by window_size
        x = x[..., :total_samples - (total_samples % window_size)]
    # Reshape x into (C, T, window)
    reshaped_x = x.reshape(x.shape[0], -1, window_size)

    return reshaped_x

**We first load and reshape all the data**

In [4]:
all_data = []
all_targets = []
for (data,target) in training_data:
    filtered_data =  butter_bandpass_filter(data,0.1,18,250,4)
    reshaped_data = reshape_array_into_windows(filtered_data,250,2)
    targets_flatten = target[..., :len(reshaped_data[0])].reshape(-1)
    reshaped_data = reshaped_data.reshape((-1,reshaped_data.shape[-1]))
    all_data.append(reshaped_data)
    all_targets.append(targets_flatten)
all_data = np.concatenate(all_data)
all_targets = np.concatenate(all_targets)
assert all_data.shape[0] == all_targets.shape[0]


In [5]:
print(all_data.shape)
print(all_targets.shape)

(261755, 500)
(261755,)


## Feature Extraction

**A) Three features => Amplitude, PSD, De**

In [99]:
#### 1. Amplitude:
amplitude = (np.max(all_data,-1) - np.min(all_data,-1)).reshape(-1)

In [100]:
#### 2. PSD, De:
stft_para={
        'stftn' :500,
        'fStart':[0.1],
        'fEnd'  :[18],
        'fs'    :250,
        'window':2,
    }

def DE_PSD(data,stft_para):
    '''
    compute DE and PSD
    --------
    input:  data [n*m]          n electrodes, m time points
            stft_para.stftn     frequency domain sampling rate
            stft_para.fStart    start frequency of each frequency band
            stft_para.fEnd      end frequency of each frequency band
            stft_para.window    window length of each sample point(seconds)
            stft_para.fs        original frequency
    output: psd,DE [n*l*k]        n electrodes, l windows, k frequency bands
    '''
    #initialize the parameters
    STFTN=stft_para['stftn']
    fStart=stft_para['fStart']
    fEnd=stft_para['fEnd']
    fs=stft_para['fs']
    window=stft_para['window']

    WindowPoints=fs*window

    fStartNum=np.zeros([len(fStart)],dtype=int)
    fEndNum=np.zeros([len(fEnd)],dtype=int)
    for i in range(0,len(stft_para['fStart'])):
        fStartNum[i]=int(fStart[i]/fs*STFTN)
        fEndNum[i]=int(fEnd[i]/fs*STFTN)

    #print(fStartNum[0],fEndNum[0])
    n=1
    m=data.shape[0]

    #print(m,n,l)
    psd = np.zeros([n,len(fStart)])
    de = np.zeros([n,len(fStart)])
    #Hanning window
    Hlength=window*fs
    #Hwindow=hanning(Hlength)
    Hwindow= np.array([0.5 - 0.5 * np.cos(2 * np.pi * n / (Hlength+1)) for n in range(1,Hlength+1)])

    WindowPoints=fs*window
    dataNow=data[0:n]
    for j in range(0,n):
        temp=dataNow[j]
        Hdata=temp*Hwindow
        FFTdata=fft(Hdata,STFTN)
        magFFTdata=abs(FFTdata[0:int(STFTN/2)])
        for p in range(0,len(fStart)):
            E = 0
            #E_log = 0
            for p0 in range(fStartNum[p]-1,fEndNum[p]):
                E=E+magFFTdata[p0]*magFFTdata[p0]
            #    E_log = E_log + log2(magFFTdata(p0)*magFFTdata(p0)+1)
            E = E/(fEndNum[p]-fStartNum[p]+1)
            psd[j][p] = E
            de[j][p] = math.log(100*E,2)
            #de(j,i,p)=log2((1+E)^4)
    # print(np.array(psd).shape)
    return psd,de

def PSD_de (data):
    all_data_psd=[]
    all_data_de=[]
    for j in range (0,data.shape[0]):
        data_win = data[j,:]
        dd = DE_PSD(data_win,stft_para)
        ddd = np.array(dd)
        all_data_psd.append(ddd[0])
        all_data_de.append(ddd[1])
        # print(j)
    return all_data_psd,all_data_de




In [21]:
psd,de = PSD_de(all_data)
psd=np.array(psd)
psd_f = psd.reshape(-1)
de=np.array(de)
de_f = de.reshape(-1)

In [25]:
#### 3. mixture of these features
training_data_1 = pd.DataFrame({"amplitude":amplitude,"psd":psd_f,"de":de_f,"target":all_targets})

In [26]:
prop_train = 0.7
n_train = int(prop_train * len(training_data_1))
train_set_1 = training_data_1[:n_train]
val_set_1 = training_data_1[n_train:]

In [27]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

# Define the model
model = XGBClassifier(use_label_encoder=False, eval_metric="mlogloss")

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'gamma': [0, 0.1, 0.2]
}

# Set up the GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(np.array(train_set_1[["amplitude","de"]]), train_set_1["target"])

# Display the best parameters and score
print("Best parameters found: ", grid_search.best_params_)
print("Best accuracy: ", grid_search.best_score_)

Fitting 3 folds for each of 324 candidates, totalling 972 fits


c:\Users\ASUS\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:14:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best parameters found:  {'colsample_bytree': 1.0, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 300, 'subsample': 1.0}
Best accuracy:  0.873299932324754


In [31]:
# mm = grid_search.best_estimator_
prediction = grid_search.predict(np.array(val_set_1[["amplitude","de"]]))

from sklearn.metrics import cohen_kappa_score, f1_score
print(cohen_kappa_score(prediction,val_set_1["target"]))
print(f1_score(val_set_1["target"],prediction))

0.48294773600435026
0.6820099140983404


**Results for above features (XGboost Method):**<br>
Score= 63.06 % on kaggle dataset (test)

**B) Summary of the 23 Extracted Features:**<br>
Standard Deviation, Mean, Median, Interquartile Range (IQR), Minimum Amplitude, Maximum Amplitude, Amplitude-Position Difference, Kurtosis, Approximate Entropy, Permutation Entropy, SVD Entropy, Sample Entropy, Higuchi Fractal Dimension, Katz Fractal Dimension, Detrended Fluctuation Analysis (DFA), Petrosian Fractal Dimension, Skewness, Hjorth, Mobility, Hjorth Complexity, Lempel-Ziv Complexity, Zero-Crossings Count, Count of Negative Values, Count of Positive Values

In [36]:
###### 23 features

from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.stats import iqr
from scipy.signal import welch
import antropy as ant

def lziv(x):
    """Binarize the EEG signal and calculate the Lempel-Ziv complexity.
    """
    return ant.lziv_complexity(x > x.mean(), normalize=True)

results = []
x = all_data[0,:]
results.append(np.std(x))
results.append(np.mean(x))
results.append(np.median(x))
results.append(iqr(x, rng=(25,75)))
results.append(np.min(x))
results.append(np.max(x))
results.append(np.sqrt((np.max(x)-np.min(x))**2 + (np.argmax(x)-np.argmin(x))**2))
results.append(kurtosis(x))
results.append(np.apply_along_axis(ant.app_entropy, axis=0, arr=x))
results.append(np.apply_along_axis(ant.perm_entropy, axis=0, arr=x, normalize=True))
results.append(np.apply_along_axis(ant.svd_entropy, axis=0, arr=x, normalize=True))
results.append(np.apply_along_axis(ant.sample_entropy, axis=0, arr=x))
results.append(np.apply_along_axis(ant.higuchi_fd, axis=0, arr=x))
results.append(np.apply_along_axis(ant.katz_fd, axis=0, arr=x))
results.append(np.apply_along_axis(ant.detrended_fluctuation, axis=0, arr=x))
results.append(ant.petrosian_fd(x, axis=0))
results.append(skew(x))
results.append(ant.hjorth_params(x, axis=0)[0])
results.append(ant.hjorth_params(x, axis=0)[1])
results.append(np.apply_along_axis(lziv, axis=0, arr=x))
results.append(ant.num_zerocross(x, axis=0))
results.append(np.sum(x<0, axis=0))
results.append(np.sum(x>0, axis=0))
results2 = np.expand_dims(np.array(results),axis=0)
X = results2


for j in range(1,all_data.shape[0]): 
    # print(j) 
    x = all_data[j,:]
    results = []
    results.append(np.std(x))
    results.append(np.mean(x))
    results.append(np.median(x))
    results.append(iqr(x, rng=(25,75)))
    results.append(np.min(x))
    results.append(np.max(x))
    results.append(np.sqrt((np.max(x)-np.min(x))**2 + (np.argmax(x)-np.argmin(x))**2))
    results.append(kurtosis(x))
    results.append(np.apply_along_axis(ant.app_entropy, axis=0, arr=x))
    results.append(np.apply_along_axis(ant.perm_entropy, axis=0, arr=x, normalize=True))
    results.append(np.apply_along_axis(ant.svd_entropy, axis=0, arr=x, normalize=True))
    results.append(np.apply_along_axis(ant.sample_entropy, axis=0, arr=x))
    results.append(np.apply_along_axis(ant.higuchi_fd, axis=0, arr=x))
    results.append(np.apply_along_axis(ant.katz_fd, axis=0, arr=x))
    results.append(np.apply_along_axis(ant.detrended_fluctuation, axis=0, arr=x))
    results.append(ant.petrosian_fd(x, axis=0))
    results.append(skew(x))
    results.append(ant.hjorth_params(x, axis=0)[0])
    results.append(ant.hjorth_params(x, axis=0)[1])
    results.append(np.apply_along_axis(lziv, axis=0, arr=x))
    results.append(ant.num_zerocross(x, axis=0))
    results.append(np.sum(x<0, axis=0))
    results.append(np.sum(x>0, axis=0))
    results2 = np.expand_dims(np.array(results),axis=0)
    X = np.concatenate((X,results2),axis=0)

In [41]:
feat_23 = X
print("23 features :",feat_23.shape)
data_3 = [amplitude,psd_f,all_targets]
feat_3 = np.array(data_3).T
All_feat = np.hstack((feat_3,feat_23))
print("All_features: 3 + 23 features :",All_feat.shape)

23 features : (261755, 23)
All_features: 3 + 23 features : (261755, 26)


In [44]:
training_data_2=pd.DataFrame(All_feat,columns=["amplitude","psd","target","stdev","mean","median","iqr","min","max","mmd","kurt","ApEn","perm_ant","svd_ant",
                                              "sample_ant", "higuchi", "katz", "dfa", "FD", "skew", "hjorth_mobility", "hjorth_complexity", "lempel-ziv", "nzc",
                                               "neg_counts","pos_counts"])

In [45]:
prop_train = 0.7
n_train = int(prop_train * len(training_data_2))
train_set_2 = training_data_2[:n_train]
val_set_2 = training_data_2[n_train:]

In [47]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

# Define the model
model = XGBClassifier(use_label_encoder=False, eval_metric="mlogloss")

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'gamma': [0, 0.1, 0.2]
}

# Set up the GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(np.array(train_set_2[["amplitude","psd","stdev","mean","median","iqr","min","max","mmd","kurt","ApEn","perm_ant","svd_ant",
                                              "sample_ant", "higuchi", "katz", "dfa", "FD", "skew", "hjorth_mobility", "hjorth_complexity", "lempel-ziv", "nzc",
                                               "neg_counts","pos_counts"]]), train_set_2["target"])

# Display the best parameters and score
print("Best parameters found: ", grid_search.best_params_)
print("Best accuracy: ", grid_search.best_score_)


Fitting 3 folds for each of 324 candidates, totalling 972 fits


c:\Users\ASUS\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:49:41] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best parameters found:  {'colsample_bytree': 1.0, 'gamma': 0.2, 'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 200, 'subsample': 0.8}
Best accuracy:  0.8722738882703517


In [48]:
# mm = grid_search.best_estimator_
prediction = grid_search.predict(np.array(val_set_2[["amplitude","psd","stdev","mean","median","iqr","min","max","mmd","kurt","ApEn","perm_ant","svd_ant",
                                              "sample_ant", "higuchi", "katz", "dfa", "FD", "skew", "hjorth_mobility", "hjorth_complexity", "lempel-ziv", "nzc",
                                               "neg_counts","pos_counts"]]))

from sklearn.metrics import cohen_kappa_score, f1_score
print(cohen_kappa_score(prediction,val_set_2["target"]))
print(f1_score(val_set_2["target"],prediction))

0.7194095876542377
0.8123282442748092


**Results for above features (XGboost Method):**<br>
Score= 72.21 % on kaggle dataset (test)

**C) Summary of the 20 Extracted Features:**<br>

Band Power Features: 5 Absolute Power values (delta, theta, alpha, sigma, beta), 5 Relative Power values (delta, theta, alpha, sigma, beta)<br>
Spectral Entropy, Sleep Spindle Count, Delta-Theta Ratio, RMS Amplitude, Wavelet Power Features

In [49]:
########### 20 features
import numpy as np
import pywt
from scipy.signal import welch
from scipy.stats import entropy
from scipy.ndimage import gaussian_filter1d

# Example EEG signal (replace with real data)
fs = 100  # Sampling frequency (Hz)
# signal = np.random.randn(fs * 10)  # Example EEG signal (10 seconds)

# Define frequency bands for sleep analysis
sub_bands = {
    "delta": [0.5, 4],
    "theta": [4, 8],
    "alpha": [8, 12],
    "sigma": [12, 16],
    "beta": [16, 25]
}

# 1. Spectral Power Calculation in Each Frequency Band
def band_power(signal, fs, sub_bands):
    freqs, psd = welch(signal, fs=fs, nperseg=fs*2, noverlap=fs//2)
    band_powers = []
    total_power = np.trapz(psd, freqs)
    for _, (low, high) in sub_bands.items():
        idx = np.logical_and(freqs >= low, freqs < high)
        band_power = np.trapz(psd[idx], freqs[idx])
        relative_band_power = band_power / total_power
        band_powers.extend([band_power, relative_band_power])
    return np.array(band_powers)

# 2. Spectral Entropy Calculation
def spectral_entropy(signal, fs):
    freqs, psd = welch(signal, fs=fs, nperseg=fs*2, noverlap=fs//2)
    psd_norm = psd / np.sum(psd)
    return entropy(psd_norm)

# 3. Sleep Spindles (Approximation using sigma band power fluctuations)
def detect_sleep_spindles(signal, fs):
    freqs, psd = welch(signal, fs=fs, nperseg=fs*2, noverlap=fs//2)
    sigma_idx = np.logical_and(freqs >= 12, freqs < 16)
    sigma_power = psd[sigma_idx]
    smoothed_sigma = gaussian_filter1d(sigma_power, sigma=1)
    spindle_count = np.sum(smoothed_sigma > np.mean(smoothed_sigma) + 2 * np.std(smoothed_sigma))
    return spindle_count

# 4. Delta-to-Theta Ratio
def delta_theta_ratio(band_powers, sub_bands):
    delta_idx = list(sub_bands.keys()).index("delta") * 2
    theta_idx = list(sub_bands.keys()).index("theta") * 2
    delta_power = band_powers[delta_idx]
    theta_power = band_powers[theta_idx]
    return delta_power / theta_power if theta_power != 0 else 0

# 5. Root Mean Square (RMS) Amplitude
def rms_amplitude(signal):
    return np.sqrt(np.mean(signal**2))

# 6. Wavelet Decomposition and Power Calculation
def wavelet_features(signal, wavelet='db4', level=5):
    coeffs = pywt.wavedec(signal, wavelet, level=level)
    wavelet_powers = [np.sum(np.square(c)) for c in coeffs]  # Power for each level
    return np.array(wavelet_powers)



In [50]:
def sub_bands_estimate (data):
    all_feats=np.zeros((data.shape[0],20))
    fs = 250
    # Define frequency bands for sleep analysis
    sub_bands = {
        "delta": [0.5, 4],
        "theta": [4, 8],
        "alpha": [8, 12],
        "sigma": [12, 16],
        "beta": [16, 25]
    }

    for j in range (0,data.shape[0]):
        signal = data[j,:]
        # Calculate features and store in a single array
        band_powers = band_power(signal, fs, sub_bands)
        spectral_entropy_value = spectral_entropy(signal, fs)
        spindle_count = detect_sleep_spindles(signal, fs)
        dt_ratio = delta_theta_ratio(band_powers, sub_bands)
        rms_amp = rms_amplitude(signal)
        wavelet_powers = wavelet_features(signal)

        # Combine all features into a single array
        features = np.concatenate((
            band_powers,                      # Absolute and relative power for each band
            np.array([spectral_entropy_value,  # Spectral entropy
                    spindle_count,           # Sleep spindle count
                    dt_ratio,                # Delta-Theta ratio
                    rms_amp]),               # RMS Amplitude
            wavelet_powers                     # Wavelet powers
            ))
        # print(features.shape)
        all_feats[j,:] = features
        # print(j)

    return all_feats

In [6]:
all_20_features = sub_bands_estimate(all_data)
feature_46 = np.hstack((All_feat,all_20_features))
print("All_features: 3 + 23 + 20 features :",feature_46.shape)
# feature_46 = np.load('Final_feature_46.npy')  # => 3 + 23 + 20 = 46 features

In [53]:
training_data_3=pd.DataFrame(feature_46,columns=["amplitude","psd","target","stdev","mean","median","iqr","min","max","mmd","kurt","ApEn","perm_ant","svd_ant",
                                              "sample_ant", "higuchi", "katz", "dfa", "FD", "skew", "hjorth_mobility", "hjorth_complexity", "lempel-ziv", "nzc",
                                               "neg_counts","pos_counts", "AP_Delta", "RP_Delta", "AP_Theta", "RP_Theta", "AP_Alpha", "RP_Alpha", "AP_Sigma", "RP_Sigma",
                                               "AP_Beta", "RP_Beta", "Spe_Entropy", "Sleep_Spindle_Count", "Delta-to-Theta Ratio", "RMS_Amp", "WaveP_L_1", "WaveP_L_2",
                                               "WaveP_L_3", "WaveP_L_4", "WaveP_L_5","WaveP_L_6"])

In [54]:
prop_train = 0.7
n_train = int(prop_train * len(training_data_3))
train_set_3 = training_data_3[:n_train]
val_set_3 = training_data_3[n_train:]

In [57]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

# Define the model
model = XGBClassifier(use_label_encoder=False, eval_metric="mlogloss")

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'gamma': [0, 0.1, 0.2]
}

# Set up the GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(np.array(train_set_3[["amplitude","psd","stdev","mean","median","iqr","min","max","mmd","kurt","ApEn","perm_ant","svd_ant",
                                              "sample_ant", "higuchi", "katz", "dfa", "FD", "skew", "hjorth_mobility", "hjorth_complexity", "lempel-ziv", "nzc",
                                               "neg_counts","pos_counts", "AP_Delta", "RP_Delta", "AP_Theta", "RP_Theta", "AP_Alpha", "RP_Alpha", "AP_Sigma", "RP_Sigma",
                                               "AP_Beta", "RP_Beta", "Spe_Entropy", "Sleep_Spindle_Count", "Delta-to-Theta Ratio", "RMS_Amp", "WaveP_L_1", "WaveP_L_2",
                                               "WaveP_L_3", "WaveP_L_4", "WaveP_L_5","WaveP_L_6"]]), train_set_3["target"])

# Display the best parameters and score
print("Best parameters found: ", grid_search.best_params_)
print("Best accuracy: ", grid_search.best_score_)


Fitting 3 folds for each of 324 candidates, totalling 972 fits


c:\Users\ASUS\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:59:16] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best parameters found:  {'colsample_bytree': 0.8, 'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 300, 'subsample': 1.0}
Best accuracy:  0.8768419673848976


In [59]:
# mm = grid_search.best_estimator_
prediction = grid_search.predict(np.array(val_set_3[["amplitude","psd","stdev","mean","median","iqr","min","max","mmd","kurt","ApEn","perm_ant","svd_ant",
                                              "sample_ant", "higuchi", "katz", "dfa", "FD", "skew", "hjorth_mobility", "hjorth_complexity", "lempel-ziv", "nzc",
                                               "neg_counts","pos_counts", "AP_Delta", "RP_Delta", "AP_Theta", "RP_Theta", "AP_Alpha", "RP_Alpha", "AP_Sigma", "RP_Sigma",
                                               "AP_Beta", "RP_Beta", "Spe_Entropy", "Sleep_Spindle_Count", "Delta-to-Theta Ratio", "RMS_Amp", "WaveP_L_1", "WaveP_L_2",
                                               "WaveP_L_3", "WaveP_L_4", "WaveP_L_5","WaveP_L_6"]]))

from sklearn.metrics import cohen_kappa_score, f1_score
print(cohen_kappa_score(prediction,val_set_3["target"]))
print(f1_score(val_set_3["target"],prediction))

0.7445419899954566
0.8275202498292182


**Results for above features (XGboost Method):**<br>
Score= 74.25 % on kaggle dataset (test)

**D) Summary of the Extracted Features:**<br>

The code extracts a rich set of 2 features, including:<br>

Time-domain features:Peak-to-Peak (PTP).<br>
Connectivity: Phase Locking Value (PLV).


In [7]:
import numpy as np
import scipy.signal as signal
import pywt
from antropy import app_entropy, svd_entropy, perm_entropy, hjorth_params
from mne_connectivity import SpectralConnectivity

from scipy.stats import kurtosis, skew
from sklearn.feature_selection import mutual_info_regression
from scipy.signal import hilbert


def compute_plv(epoch):
    analytic_signal = hilbert(epoch)
    phase_data = np.angle(analytic_signal)
    plv = np.abs(np.mean(np.exp(1j * phase_data)))
    return plv

# Function to calculate bandpower using Welch's method
def bandpower(data, fs, bands):
    freqs, psd = signal.welch(data, fs, nperseg=256)
    bandpowers = []
    for band in bands:
        idx_band = np.logical_and(freqs >= band[0], freqs <= band[1])
        bandpowers.append(np.sum(psd[idx_band]))
    return bandpowers

# Main feature extraction function
def extract_eeg_features(eeg_data, fs=250):
    num_epochs = eeg_data.shape[0]
    all_features = []
    bands = [(0.5, 4), (4, 8), (8, 12), (12, 16), (16, 30)]  # Delta, Theta, Alpha, Sigma, Beta

    for epoch in eeg_data:
        features = []
        # Time-domain features
        features.append(np.mean(epoch))
        features.append(np.std(epoch))
        features.append(np.median(epoch))
        features.append(np.min(epoch))
        features.append(np.max(epoch))
        features.append(np.ptp(epoch))
        features.append(np.percentile(epoch, 75) - np.percentile(epoch, 25))  # IQR

        # Frequency-domain features
        psd_features = bandpower(epoch, fs, bands)
        features.extend(psd_features)

        # Time-frequency features (Wavelet)
        coeffs = pywt.wavedec(epoch, 'db4', level=5)
        for coeff in coeffs:
            features.append(np.std(coeff))

        # Nonlinear dynamics
        features.append(app_entropy(epoch))
        features.append(svd_entropy(epoch))
        features.append(perm_entropy(epoch))
        hjorth_mob, hjorth_comp = hjorth_params(epoch)
        features.extend([hjorth_mob, hjorth_comp])

        # Statistical features
        features.append(kurtosis(epoch))
        features.append(skew(epoch))

        # Phase Locking Value (PLV) using spectral connectivity
        features.append(compute_plv(epoch))

        all_features.append(features)

    return np.array(all_features)


In [ ]:
features_new = extract_eeg_features(all_data, fs=250)
# np.save('New_features',features_new)
# features_new = np.load('New_features.npy')

In [10]:
print("features_new:",features_new.shape)

features_new: (261755, 26)


In [ ]:
All_features = np.hstack((feature_46,features_new))
print("All_features:",All_features.shape)

All_features: (261755, 72)


In [12]:
training_data_4=pd.DataFrame(All_features,columns=["amplitude","de","target","stdev","mean","median","iqr","min","max","mmd","kurt","ApEn","perm_ant","svd_ant",
                                              "sample_ant", "higuchi", "katz", "dfa", "FD", "skew", "hjorth_mobility", "hjorth_complexity", "lempel-ziv", "nzc",
                                               "neg_counts","pos_counts", "AP_Delta", "RP_Delta", "AP_Theta", "RP_Theta", "AP_Alpha", "RP_Alpha", "AP_Sigma", "RP_Sigma",
                                               "AP_Beta", "RP_Beta", "Spe_Entropy", "Sleep_Spindle_Count", "Delta-to-Theta Ratio", "RMS_Amp", "WaveP_L_1", "WaveP_L_2",
                                               "WaveP_L_3", "WaveP_L_4", "WaveP_L_5","WaveP_L_6",
                                               'f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12','f13','f14','f15','f16','f17','f18',
                                               'f19','f20','f21','f22','f23','f24','f25','f26'])

In [ ]:
training_data_4

,amplitude,de,target,stdev,mean,median,iqr,min,max,mmd,...,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26
0,28600.257975,1.968339e+06,0.0,7780.162127,1245.670285,-1255.114012,12656.979714,-7129.188743,21471.069232,28604.403948,...,138.938391,11.063259,0.026126,0.240860,0.300821,0.043994,6.488830,-0.610001,0.765047,0.241511
1,7506.462109,1.079117e+11,0.0,2433.203314,-4965.798852,-5273.269850,4478.943017,-7982.901067,-476.438958,7518.091340,...,3.883160,0.945529,0.008809,0.026714,0.749798,0.005689,34.624977,-1.311260,0.304853,0.880217
2,4054.175414,4.560607e+08,0.0,1187.694358,2364.739039,2753.325355,1929.311369,-464.097984,3590.077431,4081.098172,...,3.431728,0.854115,0.007134,0.031422,0.747064,0.005910,50.729171,-0.582127,-0.797484,0.862669
3,2187.981464,2.607974e+10,0.0,695.837658,2755.580822,2819.446000,1371.305305,1432.992918,3620.974382,2235.813697,...,2.677566,0.787081,0.014333,0.018240,1.113054,0.006904,59.065314,-1.245956,-0.312038,0.968304
4,2362.447080,4.271800e+09,0.0,649.347508,8.122686,-67.534054,1065.791817,-942.069600,1420.377480,2414.571848,...,3.264891,0.767291,0.014840,0.073988,1.015574,0.006834,68.005797,-0.977942,0.350397,0.278214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261750,80323.237757,1.106412e+13,0.0,23285.711223,24638.793075,20763.977697,39903.089451,-8036.808390,72286.429367,80324.787735,...,82.854637,21.626991,0.004589,0.027262,0.215619,0.003464,182.232005,-1.118886,0.352335,0.633516
261751,92070.205153,1.434080e+11,0.0,15560.939969,-14457.243453,-13804.306760,16000.334097,-58603.355007,33466.850145,92070.345920,...,336.413473,31.077920,0.202741,0.347022,1.600936,0.102091,2.034849,1.715439,0.909529,0.715875
261752,11323.766397,3.548607e+11,0.0,3217.566353,-8460.097483,-8664.572357,5615.035970,-14028.228995,-2704.462599,11333.640827,...,86.058246,28.480440,0.041347,0.063301,2.303417,0.019797,51.039491,-1.149386,0.123152,0.932375
261753,9967.617588,1.589771e+10,0.0,2897.625283,3258.551582,3636.581786,4562.200949,-2758.055101,7209.562487,9979.556322,...,86.378044,28.956137,0.048451,0.115963,2.276615,0.021408,49.254801,-0.986462,-0.454989,0.597840


In [14]:
prop_train = 0.7
n_train = int(prop_train * len(training_data_4))
train_set_4 = training_data_4[:n_train]
val_set_4 = training_data_4[n_train:]

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

# Define the model
model = XGBClassifier(use_label_encoder=False, eval_metric="mlogloss")

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'gamma': [0, 0.1, 0.2]
}

# Set up the GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(np.array(train_set_4[["amplitude","de","stdev","mean","median","iqr","min","max","mmd","kurt","ApEn","perm_ant","svd_ant",
                                              "sample_ant", "higuchi", "katz", "dfa", "FD", "skew", "hjorth_mobility", "hjorth_complexity", "lempel-ziv", "nzc",
                                               "neg_counts","pos_counts", "AP_Delta", "RP_Delta", "AP_Theta", "RP_Theta", "AP_Alpha", "RP_Alpha", "AP_Sigma", "RP_Sigma",
                                               "AP_Beta", "RP_Beta", "Spe_Entropy", "Sleep_Spindle_Count", "Delta-to-Theta Ratio", "RMS_Amp", "WaveP_L_1", "WaveP_L_2",
                                               "WaveP_L_3", "WaveP_L_4", "WaveP_L_5","WaveP_L_6",
                                               'f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12','f13','f14','f15','f16','f17','f18',
                                               'f19','f20','f21','f22','f23','f24','f25','f26']]), train_set_4["target"])

# Display the best parameters and score
print("Best parameters found: ", grid_search.best_params_)
print("Best accuracy: ", grid_search.best_score_)


In [21]:
# import joblib
# grid_search = joblib.load('xgboost_grid_search_final_46_26_features_75%.pkl')

In [22]:
prediction = grid_search.predict(np.array(val_set_4[["amplitude","de","stdev","mean","median","iqr","min","max","mmd","kurt","ApEn","perm_ant","svd_ant",
                                              "sample_ant", "higuchi", "katz", "dfa", "FD", "skew", "hjorth_mobility", "hjorth_complexity", "lempel-ziv", "nzc",
                                               "neg_counts","pos_counts", "AP_Delta", "RP_Delta", "AP_Theta", "RP_Theta", "AP_Alpha", "RP_Alpha", "AP_Sigma", "RP_Sigma",
                                               "AP_Beta", "RP_Beta", "Spe_Entropy", "Sleep_Spindle_Count", "Delta-to-Theta Ratio", "RMS_Amp", "WaveP_L_1", "WaveP_L_2",
                                               "WaveP_L_3", "WaveP_L_4", "WaveP_L_5","WaveP_L_6", 
                                               'f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12','f13','f14','f15','f16','f17','f18',
                                               'f19','f20','f21','f22','f23','f24','f25','f26']]))

from sklearn.metrics import cohen_kappa_score, f1_score
print(cohen_kappa_score(prediction,val_set_4["target"]))
print(f1_score(val_set_4["target"],prediction))

0.7463347429393861
0.8281277676093759


In [23]:
# Retrieve the best model
best_model = grid_search.best_estimator_

# Extract feature importance
feature_importance = best_model.feature_importances_

# Convert to DataFrame and sort
importance_df = pd.DataFrame({'Feature': ["amplitude","de","stdev","mean","median","iqr","min","max","mmd","kurt","ApEn","perm_ant","svd_ant",
                                              "sample_ant", "higuchi", "katz", "dfa", "FD", "skew", "hjorth_mobility", "hjorth_complexity", "lempel-ziv", "nzc",
                                               "neg_counts","pos_counts", "AP_Delta", "RP_Delta", "AP_Theta", "RP_Theta", "AP_Alpha", "RP_Alpha", "AP_Sigma", "RP_Sigma",
                                               "AP_Beta", "RP_Beta", "Spe_Entropy", "Sleep_Spindle_Count", "Delta-to-Theta Ratio", "RMS_Amp", "WaveP_L_1", "WaveP_L_2",
                                               "WaveP_L_3", "WaveP_L_4", "WaveP_L_5","WaveP_L_6", 
                                               'f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12','f13','f14','f15','f16','f17','f18',
                                               'f19','f20','f21','f22','f23','f24','f25','f26'], 'Importance': feature_importance})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Display top features
print("Top Features:")
print(importance_df)

Top Features:
                Feature  Importance
40            WaveP_L_2    0.520885
41            WaveP_L_3    0.083918
59                  f15    0.072270
61                  f17    0.037878
60                  f16    0.031166
..                  ...         ...
51                   f7    0.001794
23           neg_counts    0.001744
21           lempel-ziv    0.001645
47                   f3    0.001577
36  Sleep_Spindle_Count    0.001336

[71 rows x 2 columns]


**Results for above features (XGboost Method):**<br>
Score= 75.32 % on kaggle dataset (test)

**Saving the model**

In [ ]:
import joblib

# Save the entire GridSearchCV object
# joblib.dump(grid_search, 'xgboost_grid_search_final_46_26_features_75%.pkl')

# Load it back
# loaded_grid_search = joblib.load('xgboost_grid_search_final.pkl')
# best_xgb_model = loaded_grid_search.best_estimator_

-----------------------------


**Creating Labels for kaggel**

In [71]:
# Let's filter the signal to improve the visualisation
from scipy.signal import butter, lfilter

def butter_bandpass(lowcut, highcut, fs, order=5):
    return butter(order, [lowcut, highcut], fs=fs, btype='band')

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y


In [72]:
# First we need to get the point that maps to a label

def reshape_array_into_windows(x, sample_rate, window_duration_in_seconds):
    """
    Reshape the data into an array of shape (C, T, window) where 'window' contains
    the points corresponding to 'window_duration' seconds of data.

    Parameters:
    x (numpy array): The input data array.
    sample_rate (int): The number of samples per second.
    window_duration_in_seconds (float): The duration of each window in seconds.

    Returns:
    reshaped_x (numpy array): The reshaped array with shape (C, T, window).
    """
    # Calculate the number of samples in one window
    window_size = int(window_duration_in_seconds * sample_rate)
    
    # Ensure the total length of x is a multiple of window_size
    total_samples = x.shape[-1]
    if total_samples % window_size != 0:
        # Truncate or pad x to make it divisible by window_size
        x = x[..., :total_samples - (total_samples % window_size)]
    # Reshape x into (C, T, window)
    reshaped_x = x.reshape(x.shape[0], -1, window_size)

    return reshaped_x


In [73]:
import os
import numpy as np
import math
import scipy.io as sio
from scipy.fftpack import fft,ifft


def DE_PSD1(data,stft_para):
    '''
    compute DE and PSD
    --------
    input:  data [n*m]          n electrodes, m time points
            stft_para.stftn     frequency domain sampling rate
            stft_para.fStart    start frequency of each frequency band
            stft_para.fEnd      end frequency of each frequency band
            stft_para.window    window length of each sample point(seconds)
            stft_para.fs        original frequency
    output: psd,DE [n*l*k]        n electrodes, l windows, k frequency bands
    '''
    #initialize the parameters
    STFTN=stft_para['stftn']
    fStart=stft_para['fStart']
    fEnd=stft_para['fEnd']
    fs=stft_para['fs']
    window=stft_para['window']

    WindowPoints=fs*window

    fStartNum=np.zeros([len(fStart)],dtype=int)
    fEndNum=np.zeros([len(fEnd)],dtype=int)
    for i in range(0,len(stft_para['fStart'])):
        fStartNum[i]=int(fStart[i]/fs*STFTN)
        fEndNum[i]=int(fEnd[i]/fs*STFTN)

    #print(fStartNum[0],fEndNum[0])
    n=data.shape[0]
    m=data.shape[1]

    #print(m,n,l)
    psd = np.zeros([n,len(fStart)])
    de = np.zeros([n,len(fStart)])
    #Hanning window
    Hlength=window*fs
    #Hwindow=hanning(Hlength)
    Hwindow= np.array([0.5 - 0.5 * np.cos(2 * np.pi * n / (Hlength+1)) for n in range(1,Hlength+1)])

    WindowPoints=fs*window
    dataNow=data[0:n]
    for j in range(0,n):
        temp=dataNow[j]
        Hdata=temp*Hwindow
        FFTdata=fft(Hdata,STFTN)
        magFFTdata=abs(FFTdata[0:int(STFTN/2)])
        for p in range(0,len(fStart)):
            E = 0
            #E_log = 0
            for p0 in range(fStartNum[p]-1,fEndNum[p]):
                E=E+magFFTdata[p0]*magFFTdata[p0]
            #    E_log = E_log + log2(magFFTdata(p0)*magFFTdata(p0)+1)
            E = E/(fEndNum[p]-fStartNum[p]+1)
            psd[j][p] = E
            de[j][p] = math.log(100*E,2)
            #de(j,i,p)=log2((1+E)^4)
    # print(np.array(psd).shape)
    
    return psd,de




In [74]:
stft_para={
        'stftn' :500,
        'fStart':[0.1],
        'fEnd'  :[18],
        'fs'    :250,
        'window':2,
    }

In [75]:
def PSD_de1 (data):
    all_data_psd=[]
    all_data_de=[]
    for j in range (0,data.shape[1]):
        data_win = data[:,j,:]
        dd = DE_PSD1(data_win,stft_para)
        ddd = np.array(dd)
        all_data_psd.append(ddd[0])
        all_data_de.append(ddd[1])
        # print(j)
    return all_data_psd,all_data_de

In [76]:
from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.stats import iqr
from scipy.signal import welch
import antropy as ant
def lziv(x):
    """Binarize the EEG signal and calculate the Lempel-Ziv complexity.
    """
    return ant.lziv_complexity(x > x.mean(), normalize=True)



In [77]:
def feature23(data):
    results = []
    X = np.zeros((data.shape[0],data.shape[1],23)) 
    for k in range(0,data.shape[0]):
        for j in range(0,data.shape[1]): 
            # print(j) 
            x = data[k,j,:]
            results = []
            results.append(np.std(x))
            results.append(np.mean(x))
            results.append(np.median(x))
            results.append(iqr(x, rng=(25,75)))
            results.append(np.min(x))
            results.append(np.max(x))
            results.append(np.sqrt((np.max(x)-np.min(x))**2 + (np.argmax(x)-np.argmin(x))**2))
            results.append(kurtosis(x))
            results.append(np.apply_along_axis(ant.app_entropy, axis=0, arr=x))
            results.append(np.apply_along_axis(ant.perm_entropy, axis=0, arr=x, normalize=True))
            results.append(np.apply_along_axis(ant.svd_entropy, axis=0, arr=x, normalize=True))
            results.append(np.apply_along_axis(ant.sample_entropy, axis=0, arr=x))
            results.append(np.apply_along_axis(ant.higuchi_fd, axis=0, arr=x))
            results.append(np.apply_along_axis(ant.katz_fd, axis=0, arr=x))
            results.append(np.apply_along_axis(ant.detrended_fluctuation, axis=0, arr=x))
            results.append(ant.petrosian_fd(x, axis=0))
            results.append(skew(x))
            results.append(ant.hjorth_params(x, axis=0)[0])
            results.append(ant.hjorth_params(x, axis=0)[1])
            results.append(np.apply_along_axis(lziv, axis=0, arr=x))
            results.append(ant.num_zerocross(x, axis=0))
            results.append(np.sum(x<0, axis=0))
            results.append(np.sum(x>0, axis=0))
            # results2 = np.expand_dims(np.array(results),axis=0)
            X[k,j,:] = np.array(results)
    # print("X:",X.shape)
    return X

In [78]:
def sub_bands_estimate (data):
    # Define frequency bands for sleep analysis
    sub_bands = {
        "delta": [0.5, 4],
        "theta": [4, 8],
        "alpha": [8, 12],
        "sigma": [12, 16],
        "beta": [16, 25]
    }

    results = []
    all_feats=np.zeros((data.shape[0],data.shape[1],20))
    fs = 250
    for k in range(0,data.shape[0]):
        for j in range(0,data.shape[1]): 
            # print(j) 
            signal = data[k,j,:]
            # Calculate features and store in a single array
            band_powers = band_power(signal, fs, sub_bands)
            spectral_entropy_value = spectral_entropy(signal, fs)
            spindle_count = detect_sleep_spindles(signal, fs)
            dt_ratio = delta_theta_ratio(band_powers, sub_bands)
            rms_amp = rms_amplitude(signal)
            wavelet_powers = wavelet_features(signal)

            # Combine all features into a single array
            features = np.concatenate((
                band_powers,                      # Absolute and relative power for each band
                np.array([spectral_entropy_value,  # Spectral entropy
                        spindle_count,           # Sleep spindle count
                        dt_ratio,                # Delta-Theta ratio
                        rms_amp]),               # RMS Amplitude
                wavelet_powers                     # Wavelet powers
                ))
            # print(features.shape)
            all_feats[k,j,:] = features
            # print(k)

    return all_feats

In [87]:
def eeg_feature (data):
    ff =np.zeros((data.shape[0],data.shape[1],3))
    print(data.shape)
    
    for j in range (0,data.shape[0]):
        data_win = data[j,:,:]
        features = extract_eeg_features(data_win, fs=250)
        ff[j,:,:] = features
        # print(j)
    return np.array(ff)

In [80]:
def compute_features_on_record(data):
    """
    We compute each of the feature for each window and each channel
    Each value of the output dict has shape (Channels,T)
    """
    filtered_data =  butter_bandpass_filter(data,0.1,18,250,4)
    reshaped_data = reshape_array_into_windows(filtered_data,250,2)
    amplitude = np.max(reshaped_data,-1) - np.min(reshaped_data,-1)
    amplitude = np.expand_dims(amplitude, axis=-1)
    
    # amplitude = np.array
    print(reshaped_data.shape)
    psd,de = PSD_de1(reshaped_data)
    psd=np.array(psd)
    psd = np.transpose(psd, [1,0,2])
    # psd_f = psd.reshape(-1)
    de=np.array(de)
    de = np.transpose(de, [1,0,2])
    # de_f = de.reshape(-1)
    feat23 = feature23(reshaped_data)
    feat20 = sub_bands_estimate(reshaped_data)
    New_feature_26 = eeg_feature(reshaped_data)
    
    print('New_feature_26: ', New_feature_26.shape)
    print('amplitude: ', amplitude.shape)
    print('psd: ', psd.shape)
    print('de: ', de.shape)
    print('feat 23: ', feat23.shape)
    print('feat 20: ', feat20.shape)
    return {"amplitude":amplitude,"de":de , "feat23":feat23, "feat20":feat20 , "feat26":New_feature_26}

def compute_predictions_on_record(data,model):
    predictions = []
    features = compute_features_on_record(data)
    # print(features.shape)
    features = np.concatenate((features["amplitude"], features["de"], features["feat23"], features["feat20"], features["feat26"]), axis=-1) 
   
    # features = features.swapaxes(0,1).swapaxes(1,2)
    print(features.shape)
    for channel in range(features.shape[0]):
        predictions.append(model.predict(features[channel]))
    return np.array(predictions)

def format_array_to_target_format(array, record_number):
    assert isinstance(record_number, int)
    assert isinstance(array, np.ndarray)
    assert len(array.shape) == 2
    assert array.shape[0] == 5
    assert set(np.unique(array)) == {0, 1}
    formatted_target = []
    for i in range(array.shape[0]):
        channel_encoding = (i + 1) * 100000
        record_number_encoding = record_number * 1000000
        for j in range(array.shape[1]):
            formatted_target.append(
                {
                    "identifier": record_number_encoding + channel_encoding + j,
                    "target": array[i, j],
                }
            )
    return formatted_target
    


In [84]:
from pathlib import Path
ROOT_TEST_PATH = Path("./test/")
test_data = {i:np.load(ROOT_TEST_PATH / f"data_{i}.npy") for i in [4,5]}
# print(test_data)
# We process each record independantly
results = []
for record_number, data in test_data.items():
    print(data.shape)
    # data = data[:,:7000]
    preds = compute_predictions_on_record(data,grid_search)
    formatted_preds = format_array_to_target_format(preds,record_number)
    results.extend(formatted_preds)
df = pd.DataFrame(results)
df.to_csv("submission_final_feat49_XGBoost.csv",index = False)

{4: array([[793361.3  , 813228.7  , 802238.06 , ...,  35368.445,  36691.67 ,
         36445.62 ],
       [958025.06 , 974041.1  , 966052.6  , ...,   9994.031,  10417.463,
         10523.32 ],
       [-15702.727, -15509.607, -14547.826, ...,  11775.019,  11764.051,
         11770.25 ],
       [773123.4  , 793500.   , 785247.9  , ...,  58967.598,  57478.914,
         56180.484],
       [953489.9  , 969822.06 , 963610.25 , ...,  21818.164,  19440.652,
         18487.932]], dtype=float32), 5: array([[-114140.05   , -116140.86   , -118074.91   , ...,   -9768.964  ,
         -10117.532  ,  -10648.729  ],
       [  55726.06   ,   51648.62   ,   39194.113  , ...,   15057.565  ,
          16710.76   ,   13350.488  ],
       [  61222.562  ,   61168.2    ,   61095.246  , ...,    1237.8694 ,
           1322.7465 ,    1264.5723 ],
       [-145987.53   , -146911.64   , -142005.45   , ...,  -24088.861  ,
         -24569.04   ,  -24885.182  ],
       [ -37343.98   ,  -40290.363  ,  -45831.684  , ..., 